In [1]:
import os
import pandas as pd
from modules.data_pipeline import read_path, get_path, export_file

export_file.export_to_csv('98', 'igv')
# read_path.read(97, 'igv')

Before filter: (4221, 29)
After filter (3828, 30)
Now processing ---> ('ICA', 'ica20240502', 'AT03Data.csv')
location_block
D    59
Name: count, dtype: int64
---->AFTER:
location_block
D    198
Name: count, dtype: int64 

location_block
D    158
Name: count, dtype: int64
---->AFTER:
location_block
D    330
Name: count, dtype: int64 

location_block
D    495
E    204
Name: count, dtype: int64
---->AFTER:
location_block
E    993
D    495
Name: count, dtype: int64 

['D', 'E'] AT03-8C3.0-2024-05-03 12:00:03 

['D'] AT03-8C3.0-2024-05-03 12:00:03 

location_block
D    355
Name: count, dtype: int64
---->AFTER:
location_block
D    684
Name: count, dtype: int64 

location_block
E    588
Name: count, dtype: int64
---->AFTER:
location_block
E    1128
Name: count, dtype: int64 

Before filter: (4174, 29)
After filter (3567, 30)
Now processing ---> ('ICA', 'ica20240502', 'AT05Data.csv')
location_block
D    115
Name: count, dtype: int64
---->AFTER:
location_block
D    262
Name: count, dtype: int64

In [ ]:
import pandas as pd

def __strip_str__(df=pd.DataFrame):
    """
    Strip leading and trailing whitespaces from column names and string columns.

    Parameters:
    DataFrame (pd.DataFrame): Input DataFrame. 

    Returns:
    pd.DataFrame: DataFrame with leading and trailing whitespaces stripped from column names and string columns.
    """
    df.columns = df.columns.str.strip()
    
    # Select columns of type 'object'
    obj_cols = df.select_dtypes(['object']).columns
    
    # Strip leading and trailing whitespaces from string columns
    df[obj_cols] = df[obj_cols].apply(lambda x: x.str.strip())

    return df

# p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\TJ20240409-0410kpi\A023_IGVData_KPI.csv"
# p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\\tpy20240409\\tpy06_IGVData.csv"
# p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\\WH20240408\宝旺8号\TD002_IGVData_KPI.csv"
# p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\TS20240411\\556_IGVData.csv"
p = "C:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\\raw\W99\dl_20240408\dlqt_2_IGVData_KPI.csv"

df = pd.read_csv(p)
df = __strip_str__(df)

display(df['vesselVisitID'].value_counts().index.tolist(), df.shape)

data = df.copy()
val_to_drop = ['None', '']
data = data[data['vesselVisitID'].isin(val_to_drop)==False]

display(data['vesselVisitID'].value_counts().index.tolist(), data.shape)

In [ ]:
p = 'c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_TJ.csv'
p = os.path.abspath(p)

df = pd.read_csv(p, encoding='utf-8-sig', index_col=0)
df.columns

In [ ]:
def lmd_get_current_task_tag(current_task=str, mission_type=str, location_ref=str):
    if (current_task != 'DSCH') & (current_task != 'LOAD'):
        if ('YARD' in location_ref and mission_type=='DELIVER') or ('QC' in location_ref and mission_type=='RECEIVE'):
            return 'DSCH'
        elif('QC' in location_ref and mission_type=='DELIVER') or ('YARD' in location_ref and mission_type=='RECEIVE'):
            return 'LOAD'      
    else:
        return current_task
    
df['current_task_tag'] = df.apply(
                    lambda x: lmd_get_current_task_tag(x['current_task'], x['mission_type'], x['location_ref']),
                    axis=1)


df.columns

# Test all weekly file

In [ ]:
import os
import pandas as pd
import numpy as np

def get_all_weekly_file():
    root = 'c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99'
    pathlist = []
    for item in os.listdir(root):
        p = os.path.join(root, item)
        if os.path.isfile(p):
            pathlist.append(p)
    return pathlist

def lmd_rtg_operation(location_ref=str, task_stage=str):
    if ('YARD' in location_ref) & (task_stage=='Waiting for Operation'):
        return 'Waiting for Operation'
    else:
        return np.NaN

li = get_all_weekly_file()

In [ ]:
for p in li:
    print(p)

    df = pd.read_csv(p, encoding='utf-8-sig', index_col=0)

    print(df.columns)
    print()

In [ ]:
df.columns


In [ ]:
df['task_stage'].value_counts().index.tolist()

In [ ]:
path = os.path.abspath('c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W17\Igv_W17_TPY.csv')

d = pd.read_csv(path, encoding='utf-8-sig')
d.columns

In [ ]:
def lambda_power_usage(soc_diff):
    if soc_diff > 0 and soc_diff<=5: return soc_diff
    elif soc_diff<=0 and soc_diff>-5: return 0
    elif soc_diff >5: return 5
    elif soc_diff<=-5: return -1


for vehicle, d2 in d.groupby('vehicle_id'):
    for cycle, data in d2.groupby('Cycle Tag'):
        data['soc_diff'] = data['soc'] - data['soc'].shift(-1)
        data['power_usage'] = data['soc_diff'].apply(lambda x: lambda_power_usage(x))
        print( data['soc_diff'].value_counts().index.tolist(),
              data['power_usage'].value_counts().index.tolist())
        print()